In [4]:
import pandas as pd

In [5]:
def load_files(mamba_path: str, smartcare_path: str) -> tuple:
    """
    Load and clean data from CSV files into Pandas DataFrames.

    Args:
    - mamba_path (str): Path to the mamba CSV file.
    - smartcare_path (str): Path to the smartcare CSV file.

    Returns:
    - tuple: A tuple containing two DataFrames (df_mamba, df_smartcare).
    """
    # Load the datasets
    df_mamba = pd.read_csv(mamba_path)
    df_smartcare = pd.read_csv(smartcare_path)

    return df_mamba, df_smartcare

# Usage
#df_mamba, df_smartcare = load_files('mamba.csv', 'smartcare.csv')


In [6]:
def standardize_and_convert(df: pd.DataFrame, date_columns: list, date_format: str = "%Y-%m-%d") -> pd.DataFrame:
    """
    Standardize date formats in specified columns and convert all values to strings in a DataFrame.

    Args:
    - df (pd.DataFrame): The DataFrame to process.
    - date_columns (list): List of column names to be treated as dates.
    - date_format (str): The format to standardize date columns to.

    Returns:
    - pd.DataFrame: The processed DataFrame with standardized dates in specified columns and string values.
    """
    # Process only the specified date columns
    for col in date_columns:
        if col in df.columns:
            try:
                # Attempt to parse dates in the column
                df[col] = pd.to_datetime(df[col], errors='coerce')
                # If conversion is successful, format the dates
                if df[col].notna().any():
                    df[col] = df[col].dt.strftime(date_format)
            except (ValueError, TypeError):
                # If conversion fails, skip the column
                pass
    
    # Convert all values to strings
    df = df.astype(str)
    
    return df

# Usage example
# date_cols = ['date_column1', 'date_column2']  # Specify your date columns here
# df_mamba, df_smartcare = load_and_clean_data('mamba.csv', 'smartcare.csv')
# df_mamba = standardize_and_convert(df_mamba, date_columns=date_cols)
# df_smartcare = standardize_and_convert(df_smartcare, date_columns=date_cols)


In [39]:
#AHD Line List
df_mamba, df_smartcare = load_files('mamba.csv', 'smartcare.csv')

selected_columns = ['PatientGUID','Cryptococcal_Meningitis']
# ['Sex', 'Weight', 'Age', 'PatientGUID', 'Height', 'date_hiv_confirmed',
#        'art_start_date', 'MonthsOnART', 'FollowUpDate', 'WHO', 'CD4',
#        'ARTDoseDays', 'ARVRegimen', 'follow_up_status', 'AdheranceLevel',
#        'IsPregnant', 'FpMethodUsed', 'CrAg', 'NutritionalStatus',
#        'FunctionalStatus', 'No_OI', 'Zoster', 'Bacterial_Pneumonia',
#        'Extra_Pulmonary_TB', 'Oesophageal_Candidiasis', 'Vaginal_Candidiasis',
#        'Mouth_Ulcer', 'Chronic_Diarrhea', 'Acute_Diarrhea',
#        'CNS_Toxoplasmosis', 'Cryptococcal_Meningitis', 'Kaposi_Sarcoma',
#        'Cervical_Cancer', 'Pulmonary_TB', 'Oral_Candidiasis',
#        'Pneumocystis_Pneumonia', 'NonHodgkins_Lymphoma', 'Genital_Ulcer',
#        'OI_Other','CotrimoxazoleStartDate',
#        'cortimoxazole_stop_date', 'Fluconazole_Start_Date',
#        'Fluconazole_End_Date', 'TPT_Type', 'inhprophylaxis_started_date',
#        'InhprophylaxisCompletedDate', 'TPT_DoseDaysNumberALT',
#        'TPT_DoseDaysNumberINH', 'TPT_Dispensed_Dose', 'TPT_SideEffect',
#        'TPT_Adherence', 'tb_screened', 'tb_screening_result',
#        'TB_Diagnostic_Result', 'LF_LAM_result', 'Gene_Xpert_result',
#        'Smear_Microscopy_Result', 'Additional_TB_Diagnostic_Test_Result',
#        'Active_TB', 'ActiveTBTreatmentStartDate',
#        'ActiveTBTreatmentCompletedDate', 'ActiveTBTreatmentDiscontinuedDate',
#        'viral_load_perform_date', 'viral_load_status', 'viral_load_count',
#        'VL_Sent_Date', 'viral_load_ref_date', 'CCA_Screened', 'DSD_Category',
#        'AHD', 'Id', 'PatientId']
date_cols = ['date_hiv_confirmed','art_start_date','FollowUpDate','CotrimoxazoleStartDate','ActiveTBTreatmentStartDate'
'cortimoxazole_stop_date','Fluconazole_Start_Date','Fluconazole_End_Date','inhprophylaxis_started_date','InhprophylaxisCompletedDate',
            'ActiveTBTreatmentCompletedDate','ActiveTBTreatmentDiscontinuedDate','viral_load_perform_date','viral_load_ref_date','VL_Sent_Date']

#Convert and standardize Date columns
df_mamba = standardize_and_convert(df_mamba, date_columns=date_cols)
df_smartcare = standardize_and_convert(df_smartcare, date_columns=date_cols)

#Keep selected columns
df1 = df_mamba[selected_columns]
df2 = df_smartcare[selected_columns]


df1, df2 = df1.set_index('PatientGUID').align(df2.set_index('PatientGUID'))


# Initialize the DataFrame for differences
diff_df = pd.DataFrame(index=df1.index)

comparision_columns = ['smartcare','mamba']
# Populate the DataFrame with 'identifier' values
for column in df1.columns:
    diff_df[(column, comparision_columns[1])] = df1[column]
    diff_df[(column, comparision_columns[0])] = df2[column]

# Flatten the multi-level column index for display purposes
diff_df.columns = pd.MultiIndex.from_tuples(diff_df.columns)
diff_df.dropna(how='all', inplace=True)


# Create conditions for rows where all 'self' and 'other' values are equal or both are null
all_equal_condition = (diff_df.xs(comparision_columns[0], axis=1, level=1) == diff_df.xs(comparision_columns[1], axis=1, level=1)).all(axis=1)
both_null_condition = (diff_df.xs(comparision_columns[0], axis=1, level=1).isna() & diff_df.xs(comparision_columns[1], axis=1, level=1).isna()).all(axis=1)

# Filter out rows where all values are the same or both are null diff_df_filtered = diff_df[~(all_equal_condition | both_null_condition)]
diff_df = diff_df[~(all_equal_condition | both_null_condition)]
# Sort and save to CSV
diff_df.sort_index(inplace=True)
diff_df.to_csv("final.csv")


In [ ]:
#DataSheet_VL_Test_Received_Line_List
df_mamba, df_smartcare = load_files('mamba.csv', 'smartcare.csv')

selected_columns = ['PatientGUID','viral_load_ref_date']
# selected_columns = ['sex', 'Weight', 'age', 'date_hiv_confirmed', 'art_start_date',
#        'FollowUpDate', 'IsPregnant', 'BreastFeeding', 'ARVDispendsedDose',
#        'ARVRegimenLine', 'art_dose', 'next_visit_date', 'follow_up_status',
#        'art_dose_End', 'viral_load_perform_date', 'viral_load_test_status',
#        'viral_load_count', 'viral_load_ref_date', 'ReasonForVLTest',
#        'PMTCT_ART', 'PatientGUID']  # IsPregnant ARVRegimenLine
date_cols = ['date_hiv_confirmed','art_start_date','FollowUpDate','next_visit_date',
'art_dose_End','viral_load_perform_date','viral_load_ref_date']

#Convert and standardize Date columns
df_mamba = standardize_and_convert(df_mamba, date_columns=date_cols)
df_smartcare = standardize_and_convert(df_smartcare, date_columns=date_cols)

#Keep selected columns
df1 = df_mamba[selected_columns]
df2 = df_smartcare[selected_columns]


df1, df2 = df1.set_index('PatientGUID').align(df2.set_index('PatientGUID'))


# Initialize the DataFrame for differences
diff_df = pd.DataFrame(index=df1.index)

comparision_columns = ['smartcare','mamba']
# Populate the DataFrame with 'identifier' values
for column in df1.columns:
    diff_df[(column, comparision_columns[1])] = df1[column]
    diff_df[(column, comparision_columns[0])] = df2[column]

# Flatten the multi-level column index for display purposes
diff_df.columns = pd.MultiIndex.from_tuples(diff_df.columns)
diff_df.dropna(how='all', inplace=True)


# Create conditions for rows where all 'self' and 'other' values are equal or both are null
all_equal_condition = (diff_df.xs(comparision_columns[0], axis=1, level=1) == diff_df.xs(comparision_columns[1], axis=1, level=1)).all(axis=1)
both_null_condition = (diff_df.xs(comparision_columns[0], axis=1, level=1).isna() & diff_df.xs(comparision_columns[1], axis=1, level=1).isna()).all(axis=1)

# Filter out rows where all values are the same or both are null diff_df_filtered = diff_df[~(all_equal_condition | both_null_condition)]
diff_df = diff_df[~(all_equal_condition | both_null_condition)]
# Sort and save to CSV
diff_df.sort_index(inplace=True)
diff_df.to_csv("final.csv")


In [ ]:
#TX_CURR

In [ ]:
selected_columns = ['PatientGUID','dsd_category']
# TX CURR LINE LIST 

# df_smartcare['dsd_category'].replace('KP_DSD', 'DSD for key populations', inplace=True)
# df_smartcare['dsd_category'].replace('HEP-CAG(Health Extension Professional Led Community Based Group)', 'Health extension professional led community', inplace=True)
# df_smartcare['dsd_category'].replace('DSD for MCH', 'DSD for maternal child health', inplace=True)
# df_smartcare['dsd_category'].replace('Appointment Spacing Model (ASM)/6MMD', 'Appointment spacing model / 6MMD', inplace=True)
# df_smartcare['dsd_category'].replace('AHD (Advanced HIV Disease Model)', 'Advanced HIV disease model', inplace=True)
# df_smartcare['dsd_category'].replace('PCAD (Community Based Group model led by Peer) ', 'Community based group model by peer', inplace=True)
# selected_columns = ['sex', 'Weight', 'Age', 'FollowUpDate_GC',
#        'Next_visit_Date_GC', 
#        'ARTDoseDays', 'FollowUpStatus', 'ARTDoseEndDate_GC',
#        'AdherenceLevel', 'ARTStartDate_GC', 'FP_Status',
#        'TB_SreeningStatus', 'ActiveTBDiagnosed', 'NutritionalScrenningStatus',
#        'SexForNutrition', 'TherapeuticFoodProvided', 'PatientGUID',
#        'IsPregnant', 'BreastFeeding',  'LMP_Date_GC', 'MonthsOnART',
#        'ChildDisclosueStatus', 'dsd_category']  
# MonthsOnART - calc diff (roudup by one)
date_cols = ['FollowUpDate_GC','ARTDoseEndDate_GC','ARTStartDate_GC','LMP_Date_GC',
'Next_visit_Date_GC',]